In [4]:
import face_recognition
import cv2
import numpy as np
import pandas as pd
import datetime
import time
import os

In [2]:
def train():
# Load face dataset
face_dataset_path = "face dataset/"
face_dataset = []
id_to_name = {}
for file_name in os.listdir(face_dataset_path):
    name, _ = file_name.split("_")
    id_to_name[len(face_dataset)] = name
    image_path = os.path.join(face_dataset_path, file_name)
    image = face_recognition.load_image_file(image_path)
    face_encoding = face_recognition.face_encodings(image)[0]
    face_dataset.append(face_encoding)
    

SyntaxError: invalid syntax (443888093.py, line 1)

In [5]:

# Load face dataset
face_dataset_path = "face dataset/"
face_dataset = []
id_to_name = {}
for file_name in os.listdir(face_dataset_path):
    name, _ = file_name.split("_")
    id_to_name[len(face_dataset)] = name
    image_path = os.path.join(face_dataset_path, file_name)
    image = face_recognition.load_image_file(image_path)
    face_encoding = face_recognition.face_encodings(image)[0]
    face_dataset.append(face_encoding)

# Create attendance dataframe
col_names = ['Id', 'Name', 'Date', 'Time']
attendance = pd.DataFrame(columns=col_names)

# Open the default camera device (0) for video capture
camera = cv2.VideoCapture(0)

# Loop over each frame from the camera
while True:
    # Read the frame from the camera
    ret, frame = camera.read()
    if not ret:
        print("Error reading frame from camera")
        break

    # Convert the frame to RGB
    rgb_frame = frame[:, :, ::-1]

    # Detect faces in the frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Loop over each face in the frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Recognize the user ID for the face using the face dataset
        matches = face_recognition.compare_faces(face_dataset, face_encoding)
        individual_id = None
        for i, match in enumerate(matches):
            if match:
                individual_id = i
                break

        # Get the user name for the recognized user ID from the dictionary
        if individual_id is not None:
            individual_name = id_to_name[individual_id]
            confidence = 100
        else:
            individual_name = "Unknown"
            confidence = 0

        # Mark attendance for the recognized user
        ts = time.time()
        date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
        timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
        attendance.loc[len(attendance)] = [individual_id, individual_name, date, timeStamp]

        # Display the name label and bounding box around the face
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.75
        thickness = 2
        label = f"{individual_name} ({confidence:.0f}%)"
        color = (0, 255, 0) if individual_id is not None else (0, 0, 255)
        cv2.putText(frame, label, (left, top - 10), font, font_scale, color, thickness)
        cv2.rectangle(frame, (left, top), (right, bottom), color, thickness)

        # Save attendance record to CSV file
        attendance.to_csv('Attendance.csv', index=False)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Exit loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the window
camera.release()
cv2.destroyAllWindows()

In [6]:
os.getcwd()

'C:\\Users\\Q209\\Current projects\\Face Recognition_FR'